In [10]:
import pandas as pd
import numpy as np
import nltk as nk
import re 

In [11]:
from itertools import chain

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
#df = pd.read_excel("Data-FakeRealCOVID.xlsx")
df = pd.read_excel("/content/drive/MyDrive/PROJET_IA/Data-FakeRealCOVID.xlsx")

In [14]:
df

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real
...,...,...,...
6415,6416,A tiger tested positive for COVID-19 please st...,fake
6416,6417,???Autopsies prove that COVID-19 is??� a blood...,fake
6417,6418,_A post claims a COVID-19 vaccine has already ...,fake
6418,6419,Aamir Khan Donate 250 Cr. In PM Relief Cares Fund,fake


**Hamronization**

In [15]:
def lowerTweet(tweet):
    return tweet.lower()

In [16]:
df.tweet.loc[0]

'The CDC currently reports 99031 deaths. In general the discrepancies in death counts between different sources are small and explicable. The death toll stands at roughly 100000 people today.'

In [17]:
lowerTweet(df.tweet.loc[0])

'the cdc currently reports 99031 deaths. in general the discrepancies in death counts between different sources are small and explicable. the death toll stands at roughly 100000 people today.'

**Remove URLS**

In [18]:
# In order to remove any URL within a string in Python, you can use this RegEx function :
def removeURLs(tweet):
        return re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", tweet)


In [19]:
df.loc[4,'tweet']

'Populous states can generate large case counts but if you look at the new cases per million today 9 smaller states are showing more cases per million than California or Texas: AL AR ID KS KY LA MS NV and SC. https://t.co/1pYW6cWRaS'

In [21]:
removeURLs(df.loc[4,'tweet'])

'Populous states can generate large case counts but if you look at the new cases per million today 9 smaller states are showing more cases per million than California or Texas: AL AR ID KS KY LA MS NV and SC.  '

**Remove emojis**

In [22]:
#i took an example to work on 
example = df.loc[89,'tweet']
example

'📢#CoronaVirusUpdates: 📍#COVID19 India Tracker (As on 7 September 2020 08:00 AM) ➡️Confirmed cases: 4204613 ➡️Recovered: 3250429 (77.3%)👍 ➡️Active cases: 882542 (21.0%) ➡️Deaths: 71642 (1.7%) #IndiaFightsCorona #IndiaWillWin #StaySafe Via @MoHFW_INDIA https://t.co/WVQslxqBPC'

In [25]:
# Remove all traces of emoji from a text file.

'''def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)
remove_emoji(df.loc[6416,'tweet'])'''

'def remove_emoji(string):\n    emoji_pattern = re.compile("["\n                               u"😀-🙏"  # emoticons\n                               u"🌀-🗿"  # symbols & pictographs\n                               u"🚀-\U0001f6ff"  # transport & map symbols\n                               u"\U0001f1e0-🇿"  # flags (iOS)\n                               u"─-⯯"  # chinese char\n                               u"✂-➰"\n                               u"✂-➰"\n                               u"Ⓜ-🉑"\n                               u"🤦-🤷"\n                               u"𐀀-\U0010ffff"\n                               u"♀-♂"\n                               u"☀-⭕"\n                               u"\u200d"\n                               u"⏏"\n                               u"⏩"\n                               u"⌚"\n                               u"️"  # dingbats\n                               u"〰"\n                               "]+", flags=re.UNICODE)\n    return emoji_pattern.sub(r\'\', string)\nremov

In [26]:
#the easiet solution we found is to turn text incode into ascii wich does not include emojis and ignore all other characters that are not defined on ascii encode
#we use encode function which turns string into ascii code and ignoring none ascii characters
#then decode to make the ascii code a string again
def toAscii(tweet):
  return  (tweet.encode('ascii',errors='ignore')).decode('ascii') 


In [27]:
toAscii(example)

'#CoronaVirusUpdates: #COVID19 India Tracker (As on 7 September 2020 08:00 AM) Confirmed cases: 4204613 Recovered: 3250429 (77.3%) Active cases: 882542 (21.0%) Deaths: 71642 (1.7%) #IndiaFightsCorona #IndiaWillWin #StaySafe Via @MoHFW_INDIA https://t.co/WVQslxqBPC'

**Remove ponctuation**

In [28]:
#removing all numbers ,ponctuation & special characters such as @ and #
#refrence to https://www.kerryr.net/pioneers/ascii3.htm

def removeSpecialChar(text):
    text = list(text)
    
    #remove special character & ponctuations & numbers
    listOfSpecialChars =  [*range(33,48),*range(58,64),*range(91,97),*range(123,127)]
    
    text = [char for char in text if (ord(char) not in listOfSpecialChars and not char.isdigit())] 
    
    return ''.join(text)

In [29]:
#lets test on this example
df.loc[51].tweet

'132 new cases of #COVID19Nigeria; Lagos-52 Gombe-27 Plateau-17 Kwara-10 Enugu-9 Ogun-9 Katsina-3 Ekiti-2 Bauchi-1 Osun-1 Rivers-1 56388 confirmed 44337 discharged 1083 deaths https://t.co/Enpey5u7xh'

In [30]:
removeSpecialChar(df.loc[51].tweet)

' new cases of COVIDNigeria Lagos Gombe Plateau Kwara Enugu Ogun Katsina Ekiti Bauchi Osun Rivers  confirmed  discharged  deaths httpstcoEnpeyuxh'

**Removing stopWords**

In [31]:
nk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [53]:
def removeStopWords(text):
    sw = stopwords.words("english")
    return removeSpecialChar(text)

In [33]:
df.loc[51].tweet

'132 new cases of #COVID19Nigeria; Lagos-52 Gombe-27 Plateau-17 Kwara-10 Enugu-9 Ogun-9 Katsina-3 Ekiti-2 Bauchi-1 Osun-1 Rivers-1 56388 confirmed 44337 discharged 1083 deaths https://t.co/Enpey5u7xh'

In [54]:
removeStopWords(df.loc[51].tweet)

' new cases of COVIDNigeria Lagos Gombe Plateau Kwara Enugu Ogun Katsina Ekiti Bauchi Osun Rivers  confirmed  discharged  deaths httpstcoEnpeyuxh'

**Data** **Preprocessing**:

In [35]:

# def preprocess(text):
  # lowercase the text
  # text=text.lower()
  # remove the words counting just one letter
  # text = [t for t in text.split(" ") if len(t) > 1]
  # text = re.sub('https?://\S+|www\.\S+', '', text)
  # return text


In [36]:
# df = pd.read_excel("/content/drive/MyDrive/PROJET_IA/Data-FakeRealCOVID.xlsx")

# df['tweet']=df['tweet'].apply(preprocess)

In [55]:
def preprocess(text):
    text = lowerTweet(text)
    text = removeURLs(text)
    text = toAscii(text)
    text = removeSpecialChar(text)
    text = removeStopWords(text)
    return text

In [51]:
df.loc[209,'tweet']

'Children under 18 years represent about 8.5% of reported #COVID19 cases with few deaths compared to other age groups &amp; usually mild disease. 👉 https://t.co/Cs7uxQRRMN https://t.co/l4gqfttYmg'

In [56]:
preprocess(df.loc[209,'tweet'])

'children under  years represent about  of reported covid cases with few deaths compared to other age groups amp usually mild disease     '

**Tokenize**

In [40]:
nk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [41]:
def tokenize(text):
    return nk.word_tokenize(text)

In [46]:
df.loc[51,'tweet']

'132 new cases of #COVID19Nigeria; Lagos-52 Gombe-27 Plateau-17 Kwara-10 Enugu-9 Ogun-9 Katsina-3 Ekiti-2 Bauchi-1 Osun-1 Rivers-1 56388 confirmed 44337 discharged 1083 deaths https://t.co/Enpey5u7xh'

In [57]:
tokenize(removeSpecialChar(df.loc[51,'tweet']))

['new',
 'cases',
 'of',
 'COVIDNigeria',
 'Lagos',
 'Gombe',
 'Plateau',
 'Kwara',
 'Enugu',
 'Ogun',
 'Katsina',
 'Ekiti',
 'Bauchi',
 'Osun',
 'Rivers',
 'confirmed',
 'discharged',
 'deaths',
 'httpstcoEnpeyuxh']

In [58]:
def bagOfWords(df):
    bag = []
    for tweet in df.tweet:
        bag += tokenize(preprocess(tweet))
    return bag

In [59]:
bagOfWords(df)

['the',
 'cdc',
 'currently',
 'reports',
 'deaths',
 'in',
 'general',
 'the',
 'discrepancies',
 'in',
 'death',
 'counts',
 'between',
 'different',
 'sources',
 'are',
 'small',
 'and',
 'explicable',
 'the',
 'death',
 'toll',
 'stands',
 'at',
 'roughly',
 'people',
 'today',
 'states',
 'reported',
 'deaths',
 'a',
 'small',
 'rise',
 'from',
 'last',
 'tuesday',
 'southern',
 'states',
 'reported',
 'of',
 'those',
 'deaths',
 'politically',
 'correct',
 'woman',
 'almost',
 'uses',
 'pandemic',
 'as',
 'excuse',
 'not',
 'to',
 'reuse',
 'plastic',
 'bag',
 'coronavirus',
 'nashville',
 'indiafightscorona',
 'we',
 'have',
 'covid',
 'testing',
 'laboratories',
 'in',
 'india',
 'and',
 'as',
 'on',
 'th',
 'august',
 'tests',
 'have',
 'been',
 'done',
 '@',
 'profbhargava',
 'dg',
 '@',
 'icmrdelhi',
 'staysafe',
 'indiawillwin',
 'populous',
 'states',
 'can',
 'generate',
 'large',
 'case',
 'counts',
 'but',
 'if',
 'you',
 'look',
 'at',
 'the',
 'new',
 'cases',
 'per',

In [ ]:

# def preprocess(text):
#   # lowercase the text
#   text=text.lower()
#   # remove the words counting just one letter
#   text = [t for t in text.split(" ") if len(t) > 1]
#   # text = re.sub('https?://\S+|www\.\S+', '', text)
#   # text =re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", text )
#   # Remove all traces of emoji from a text file.
#   # text= remove_emoji(text)
#   text = [ps.stem(word) for word in text if not word in stopwords.words('english')]
#   return text



In [ ]:
# !pip install nltk
# import nltk
# import re
# from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer
# df = pd.read_excel("/content/drive/MyDrive/PROJET_IA/Data-FakeRealCOVID.xlsx")
# # 
# df['tweet']=df['tweet'].apply(preprocess)

In [ ]:
df.head()